### Adapters

* https://github.com/dashea/requests-file
* http://docs.python-requests.org/en/master/user/advanced/#transport-adapters
* https://github.com/reclosedev/requests-cache

In [1]:
    import traitlets, requests, requests_cache, IPython, requests_file

In [2]:
    class Adapter(traitlets.HasTraits):
        protocol = traitlets.Unicode()

In [3]:
    class FileAdapter(Adapter, requests_file.FileAdapter):
        protocol = traitlets.Unicode('file://')
        def send(Adapter, request, **kwargs):
            response = super().send(request, **kwargs)
            response.request = request
            return response

In [4]:
    class HttpCache(Adapter, requests.adapters.HTTPAdapter):
        protocol = traitlets.Unicode('http:// https://')
        def __init__(HttpCache, cache_name, **kwargs):
            HttpCache.cache_name = cache_name
            super().__init__(**kwargs)
            
        def send(HttpCache, request, **kwargs):
            try:
                requests_cache.install_cache(HttpCache.cache_name)
                return requests.Session().send(request, **kwargs)
            finally: requests_cache.uninstall_cache()

In [5]:
    import notebook.services.contents.largefilemanager

    import tornado.web

    import requests

In [23]:
    class Noop(Adapter, requests.adapters.BaseAdapter):
        protocol = traitlets.Unicode('noop://')
        def send(self, request, **kwargs):
            request.url = request.url.partition('://')[2]
            response = requests.Response()
            response.request = request
            response.url = request.url
            response.status_code = 200
            return response

In [22]:
    def response_from_contents(contents_manager, response):
        """
    >>> contents_manager = notebook.services.contents.largefilemanager.LargeFileManager()
    >>> response = requests.Response(); response.url = '/__init__.py'
    >>> response = response_from_contents(contents_manager, response)
    >>> response.headers, response.status_code
    ({'Content-Type': 'text/x-python'}, 200)
    >>> response = requests.Response(); response.url = '/__init__.p'
    >>> response = response_from_contents(contents_manager, response)
    >>> response.status_code
    404
    """
        try:
            object = contents_manager.get(response.url)
            response.status_code = 200
            response._content = object['content']
            response.headers = {'Content-Type': object['mimetype'],}
            response.size = object['size']
            response.created = object['created']
            response.last_modified = object['last_modified']
        except tornado.web.HTTPError as Exception: 
            response.status_code = Exception.status_code
        return response

In [24]:
    class ContentsAdapter(Noop):
        """
    >>> ContentsAdapter().send(requests.Request('GET', '/__init__.p'))
    <Response [404]>
    >>> content = ContentsAdapter().send(requests.Request('GET', '/__init__.py'))
    >>> content
    <Response [200]>

        """
        protocol = traitlets.Unicode('contents://')
        contents_manager = traitlets.Instance(notebook.services.contents.largefilemanager.LargeFileManager, tuple())
        def send(ContentsAdapter, request, **kwargs):
            response = super().send(request)
            return response_from_contents(ContentsAdapter.contents_manager, response)

In [26]:
    class PythonAdapter(Noop):
        protocol = traitlets.Unicode('python://')
        def send(PythonAdapter, request, *current, **kwargs):
            response = super().send(request)
            response.object = None
            for key in response.url.split('/'):
                key, sep, query = key.partition('?')
                response.object = response.object is None and importlib.import_module(key) or getattr(response.object, key, importlib.import_module('.'.join(current)))
                current += key,       
            return response

In [1]:
    import git

In [26]:
    class GitAdapter(Noop):
        protocol = traitlets.Unicode('git://')